In [13]:
import cv2
import numpy as np

In [5]:
cv2.__version__

'4.0.0'

In [3]:
import os
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [11]:
# Open connection to camera
webcam = cv2.VideoCapture(0)

#External Window or in Notebook
cv2.namedWindow("opencv_1", cv2.WINDOW_NORMAL)
 
while True:
      
    ret, frame = webcam.read()
    cv2.imshow("opencv_1", frame) 
     
    # Press q for close window
    #here 10 is a milliseconds
    if cv2.waitKey(10)== ord('q'):
        break
webcam.release()        
cv2.destroyAllWindows() 

### Face Detection using Haar Cascades

List of available pretraied models in OpenCV:

<div style="float: left; width: 50%">
<ul>
    <li> haarcascade_eye_tree_eyeglasses  
    <li> haarcascade_mcs_leftear
    <li> haarcascade_eye                  
    <li> haarcascade_mcs_lefteye
    <li> haarcascade_frontalface_alt2   
    <li> haarcascade_mcs_mouth
    <li> haarcascade_frontalface_alt_tree
    <li> haarcascade_mcs_nose
    <li> <font style="color: #be2830">haarcascade_frontalface_alt</font>       
    <li> haarcascade_mcs_rightear
    <li> haarcascade_frontalface_default
</ul>
</div>
<div style="float: right; width: 50%">
<ul>
    <li> haarcascade_mcs_righteye
    <li> haarcascade_fullbody            
    <li> haarcascade_mcs_upperbody
    <li> haarcascade_lefteye_2splits    
    <li> haarcascade_profileface
    <li> haarcascade_lowerbody            
    <li> haarcascade_righteye_2splits
    <li> haarcascade_mcs_eyepair_big     
    <li> haarcascade_smile
    <li> haarcascade_mcs_eyepair_small
    <li> haarcascade_upperbody
</ul>
</div>


### Detecting Faces with OpenCV

### detector = cv2.CascadeClassifier( xml_file_path)

### face_coord = detector.detectMultiScale(image, scale_factor, min_neighbors, min_size, max_size)

#### image : gray scale image where face are detected.


#### scale_factor:Parameter specifying how much the image size is reduced at each image scale.


#### minNeighbors – Parameter specifying how many neighbors each candidate rectangle should have to retain it.


#### minSize – Minimum possible object size. Objects smaller than that are ignored.


#### maxSize – Maximum possible object size. Objects larger than that are ignored.

In [12]:
faceDetect=cv2.CascadeClassifier('xml/frontal_face.xml')
cam = cv2.VideoCapture(0)
cv2.namedWindow("Face", cv2.WINDOW_NORMAL)
#s=(70,30)
while True:
    ret,img = cam.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceDetect.detectMultiScale(gray,1.2)
    #[[x,y,w,h],[x,y,w,h]]
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        #(image,(x,y),(w,h),border_color,border_thickness)  
        
        
    cv2.imshow("Face",img)
    if (cv2.waitKey(10)== ord('q')):
        break

cam.release()
cv2.destroyAllWindows()

### Project:face Recognition

<h2 align="center" style='color: #be2830'>Build Our Dataset</h2>
<h4 align="center">
Detect $\rightarrow$ Cut $\rightarrow$ Normalize $\rightarrow$ Resize $\rightarrow$ Save</h4>

### detect face

In [4]:
def detect_face(frame):
   
        
    detector = cv2.CascadeClassifier("xml/frontal_face.xml")

    faces = detector.detectMultiScale(frame,1.2)
    
    return faces

In [5]:
def gray_scale(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image

### Cut face

In [6]:
def cut_faces(image, faces_coord):
    faces = []
      
    for (x, y, w, h) in faces_coord:
        
        faces.append(image[y: y + h, x : x + w ])
         
    return faces

### Normalize faces by increasing pixel intensity(brightness)

In [7]:
def normalize_intensity(images):
    images_norm = []
    for image in images:
        images_norm.append(cv2.equalizeHist(image))
    return images_norm

### Resize

#### cv.INTER_AREA for shrinking & cv.INTER_CUBIC for zooming

In [8]:
def resize(images,size=(47,62)):
    image_resize = []
    
    for image in images:
        img_size = cv2.resize(image,size)
        
        image_resize.append(img_size)
        
    return image_resize

### Make pipeline

In [17]:
def normalize_faces(frame, faces_coord):
    #gray_frame = gray_scale(frame)
    faces = cut_faces(frame, faces_coord)
    faces = normalize_intensity(faces)
    
    faces = resize(faces)
    return faces

In [18]:
def plot_show(image,title=""):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.title(title)
    plt.imshow(image,cmap="Greys_r")
    plt.show()

In [19]:
def draw_rectangle(image, coords):
    for (x, y, w, h) in coords:
        
        cv2.rectangle(image, (x , y), (x + w , y + h), (0,0,255),2)

In [20]:
cam = cv2.VideoCapture(0)
#cv2.namedWindow("Face", cv2.WINDOW_AUTOSIZE)
folder = "user/"+input('Person:').lower()


if not os.path.exists(folder):
    os.mkdir(folder)
    
    flag_start_capturing = False
    sample=1
   
    cv2.namedWindow("Face", cv2.WINDOW_NORMAL)

    while True:
        ret,frame = cam.read()
        gray = gray_scale(frame)
        faces_coord = detect_face(gray)

        if len(faces_coord):
            faces = normalize_faces(gray,faces_coord)
              
            cv2.imwrite(folder + '/' + str(sample)+'.jpg',faces[0])
            plot_show(faces[0],"Image saved:"+str(sample))
            clear_output(wait=True)
            if flag_start_capturing == True:
                sample += 1
            
        draw_rectangle(frame,faces_coord)
        cv2.imshow('Face',frame)
        keypress=cv2.waitKey(1)
        
        if keypress == ord('c'):
            
            if flag_start_capturing == False:
                flag_start_capturing = True
            
        
        if sample >500:
            break
            
        if keypress == ord('q'):
            break

    cam.release()
    cv2.destroyAllWindows()
else:
    print ("This name already exists.")

Person:suhani
This name already exists.


In [21]:
cam.release()

### Create dataset for unknown people

In [22]:
### datasets for other class

basepath = 'C:\\Users\\lenovo\\scikit_learn_data\\lfw_home\\lfw_funneled\\'
#basepath='C:\\Users\lenovo\\Desktop\\material\\ML_pro\\opencv_tut\\user\\unknown\\'
##C:\Users\lenovo\scikit_learn_data\lfw_home
images = os.listdir(basepath) 
print (len(images))
data = images[:410]

for i,folder in enumerate(data,start=1):
    
    files=os.listdir(basepath+'\\'+folder)
    for k,img in enumerate(files,start=1):
        if img.endswith('.jpg'):
            #print img
            frame=cv2.imread(basepath+'\\'+folder+'\\'+img,0)
        #print frame
       
            faces_coord = detect_face(frame)
            if len(faces_coord):
                faces = cut_faces(frame, faces_coord)
                #print faces
                faces = normalize_intensity(faces)
                faces = resize(faces)
                cv2.imwrite('user/unknown/' + str(i)+'.jpg',faces[0])
                
                break

5760


### collect dataset

In [14]:
def collect_dataset():
    images = []
    labels = []
    labels_dic = {}
   
    people = [person for person in os.listdir("user/")]
   
    for i, person in enumerate(people):
        labels_dic[i] = person
        for image in os.listdir("user/" + person):
            if image.endswith('.jpg'):
                images.append(cv2.imread("user/" + person + '/' + image, 0))
                labels.append(i)
    return (images, np.array(labels), labels_dic)

In [15]:
images, labels, labels_dic = collect_dataset()

In [16]:
print (len(images))
print (labels_dic)
np.bincount(labels)

2102
{0: 'dj', 1: 'suhani', 2: 'unknown', 3: 'vineet'}


array([203, 500, 408, 991], dtype=int64)

In [23]:
X_train=np.asarray(images)

In [24]:
X_train.shape

(1151, 62, 47)

In [25]:
train=X_train.reshape(len(X_train),-1)

In [26]:
train.shape

(1151, 2914)

In [27]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [28]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(train.astype(np.float64))

In [29]:
pca1 = PCA(n_components=.97)
new_train=pca1.fit_transform(X_train_sc)

In [30]:
pca1.n_components_

314

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import GridSearchCV,KFold

In [ ]:
#log = LogisticRegression()

In [ ]:
#log.fit(new_train,labels)

In [32]:
param = {'C':[.001,.01,.1,1,10],'penalty':['l2','l1']}

In [33]:
gd = GridSearchCV(LogisticRegression(),param_grid=param,cv=5,scoring='accuracy')

In [34]:
gd.fit(new_train,labels)

c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", F

c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", F

c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", F

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'penalty': ['l2', 'l1']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [35]:
gd.best_score_

0.9730668983492615

In [36]:
gd.best_params_

{'C': 1, 'penalty': 'l1'}

In [37]:
clf2 = gd.best_estimator_

In [38]:
filename = 'log_face.pkl'
f=open(filename, 'wb')
pickle.dump(clf2,f)
 

f.close()

In [39]:
filename = 'log_face.pkl'


clf_new= pickle.load(open(filename, 'rb'))

In [40]:
clf_new

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [41]:
kf=KFold(n_splits=5,shuffle=True)

In [42]:
param_grid = {'C':[.0001,.001,.01,.1,1,10]}

In [43]:
gs_svc = GridSearchCV(SVC(kernel='linear',probability=True),param_grid=param_grid,cv=kf,scoring='accuracy')

In [44]:
gs_svc.fit(new_train,labels)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [45]:
gs_svc.best_score_

0.9774109470026064

In [46]:
gs_svc.best_params_

{'C': 0.01}

In [47]:
clf_svc=gs_svc.best_estimator_

In [48]:
import pickle

In [49]:
filename = 'svc_linear_face.pkl'
f=open(filename, 'wb')
pickle.dump(clf_svc,f)
 

f.close()


In [50]:
filename = 'svc_linear_face.pkl'
svc1 = pickle.load(open(filename, 'rb'))

In [51]:
svc1

SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [52]:
cam = cv2.VideoCapture(0)
font=cv2.FONT_HERSHEY_PLAIN
cv2.namedWindow("opencv_face", cv2.WINDOW_AUTOSIZE)



while True:
    ret,frame = cam.read()
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces_coord = detect_face(gray) # detect more than one face
    if len(faces_coord):
        faces = normalize_faces(gray, faces_coord)
        #faces = normalize_intensity(faces)
        for i, face in enumerate(faces): # for each detected face
            
            
            #cv2.imwrite('trainingData/female/picture_BGR5.jpg',face)
            t=face.reshape(1,-1)
            t=sc.transform(t.astype(np.float64))
            test = pca1.transform(t)    
            #print test
            #transform = test.reshape(1,-1)
            #print transform
            prob=svc1.predict_proba(test)
            confidence = svc1.decision_function(test)
            print (confidence)
            print (prob)
           
            
            
            pred = svc1.predict(test)
            print (pred,pred[0])
           
            name=labels_dic[pred[0]].capitalize()
            print (name)
            
            #pred = labels_dic[pred[0]].capitalize()
            #threshold = .50
            
            
                
            #cv2.putText(frame,name,(faces_coord[i][0], faces_coord[i][1] - 10),
             #          cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
            
                
           
            if prob[0][1]>.9:
                
                cv2.putText(frame, 'suhani',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
            
                
            elif prob[0][0]>.85:
                cv2.putText(frame,'unknown',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
                
                
           
        clear_output(wait = True)
        draw_rectangle(frame, faces_coord) # rectangle around face
        
    cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2,cv2.LINE_AA)
    
    cv2.imshow("opencv_face", frame) # live feed in external
    if cv2.waitKey(5) == 27:
        break
        
cam.release()
cv2.destroyAllWindows()
        

[[ 2.03945263 -0.25975188  3.5         0.72029925]]
[[0.05787064 0.00920819 0.92989042 0.00303075]]
[2] 2
Unknown


In [ ]:
cam.release()